In [1]:
import numpy as np
import os
import scipy.io as io

In [2]:
directory_asd = 'shareddata/3Dmat4ML/Autism/'
directory_typical = 'shareddata/3Dmat4ML/Neurotypical/'

In [3]:
list_dir_asd = list(os.listdir(directory_asd))
list_dir_typical = list(os.listdir(directory_typical))
n_asd = len(list_dir_asd) 
n_typical = len(list_dir_typical) 
dat_asd = np.zeros(len(list_dir_asd), dtype=object)
dat_typical = np.zeros(len(list_dir_typical), dtype=object)
i = 0
for filename in list_dir_asd:
    dat_asd[i] = io.loadmat(directory_asd+filename)
    i+=1
i = 0
for filename in list_dir_typical:
    dat_typical[i] = io.loadmat(directory_typical+filename)
    i+=1

In [4]:
"""
Saving conditions separately and labeling asd vs typical 
groups are here to perform cross-validation by excluding participants
and not trial at random: we like te predict asd or not for individuals.
"""
ERPs = np.zeros((4), dtype=object)
groups = np.zeros((4), dtype=object) 
asd = np.zeros((4), dtype=object)

for i in range(4):
    for j in range(n_asd):
        conds = dat_asd[j]['evConds'][0]==i+1
        if j == 0:
            ERPs[i] = dat_asd[j]['ERPs'][:,:,conds]
            groups[i] = j*np.ones(conds.sum())
            asd[i] = np.ones(conds.sum())
        else:
            ERPs[i] = np.concatenate((ERPs[i],dat_asd[j]['ERPs'][:,:,conds]),
                                     axis=2)
            groups[i] = np.concatenate((groups[i],j*np.ones(conds.sum())))
            asd[i] = np.concatenate((asd[i], np.ones(conds.sum())))
            
    for j in range(n_typical):
        conds = dat_typical[j]['evConds'][0]==i+1
        ERPs[i] = np.concatenate((ERPs[i],dat_typical[j]['ERPs'][:,:,conds]),
                                 axis=2)
        groups[i] = np.concatenate((groups[i],(n_asd+j)*np.ones(conds.sum())))
        asd[i] = np.concatenate((asd[i], np.zeros(conds.sum())))

In [12]:
save_directory = 'shareddata/3Dmat4ML/'
np.savez(save_directory+'ERPs.npz', ERPs_C0=ERPs[0], ERPs_C1=ERPs[1], ERPs_C2=ERPs[2],
         ERPs_C3=ERPs[3], groups=groups, asd=asd)

In [11]:
# help(np.savez)